<a href="https://colab.research.google.com/github/K4m1-5am4/DataAnalysis_and_ML/blob/main/NNclassif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>